In [ ]:
import pandas as pd
import networkx as nx
from networkx.drawing.nx_pydot import write_dot

In [ ]:
# java
def process_java_dependency(input_file, output_file="Java_graph_new.dot"):
    # Initialize a directed graph
    G = nx.DiGraph()

    # Read the CSV and add edges to the graph
    with open(input_file, 'r') as file:
        for line in file:
            # Remove spaces and split the line into source and target
            line = line.strip()

            # Check for malformed lines or invalid data
            if "->" in line:
                source, target = line.split("->")
                # Filter out edges where the target is invalid
                if target.lower() not in ['nan', 'none', '']:
                    G.add_edge(source.strip(), target.strip())
                else:
                    print(f"Skipping invalid line (nan or None target): {line}")
            else:
                print(f"Skipping malformed line: {line}")

    # Write the graph to a DOT file
    write_dot(G, output_file)
    print(f"DOT file saved to {output_file}")

def make_java_graph(df, min_edges=0):
    java_DG = nx.DiGraph()
    df['from'] = df['from'].astype(str).str.strip()
    df['to'] = df['to'].astype(str).str.strip()
    df = df.dropna()
    nodes = pd.concat([df['from'], df['to']]).unique()
    edges = df[['from', 'to']].values
    java_DG.add_nodes_from(nodes)
    java_DG.add_edges_from(edges)

    if min_edges > 0:
        to_remove = [node for node, degree in java_DG.degree() if degree <= min_edges]
        java_DG.remove_nodes_from(to_remove)

    return java_DG


In [ ]:
# python
def make_python_graph(df, min_edges=0):
    py_DG = nx.DiGraph()
    nodes = df['package_name'].dropna().unique()
    edges = df[['package_name', 'requirement']].dropna().values
    py_DG.add_nodes_from(nodes)
    py_DG.add_edges_from((edge[0], edge[1]) for edge in edges if edge[1] not in ['.', 'nan', None])
    to_remove = [node for node, degree in py_DG.degree() if degree <= min_edges]
    py_DG.remove_nodes_from(to_remove)
    return py_DG

In [ ]:
# r
def make_r_graph(df, min_edges=0):
    r_DG = nx.DiGraph()
    nodes = pd.concat([df['from'], df['to']]).dropna().unique()
    edges = df[['from', 'to']].dropna().values
    r_DG.add_nodes_from(nodes)
    r_DG.add_edges_from(edges)
    if min_edges > 0:
        to_remove = [node for node, degree in r_DG.degree() if degree <= min_edges]
        r_DG.remove_nodes_from(to_remove)
    return r_DG

In [ ]:
java_dependency = pd.read_csv( 'Javadependency.csv', delimiter='->', header=None, names=['from', 'to'], engine='python')
java_DG = make_java_graph(java_dependency, min_edges=0)
print(f"\nJava dependency graph has {java_DG.number_of_nodes()} nodes and {java_DG.number_of_edges()} edges.")
write_dot(java_DG, 'Java_graph.dot')
print("Graph has been successfully written to 'Java_graph.dot'.")

python_dependency = pd.read_csv('pythondependency.csv')
py_DG = make_python_graph(python_dependency, min_edges=0)
print(f"\nPython dependency graph has {py_DG.number_of_nodes()} nodes and {py_DG.number_of_edges()} edges.")
write_dot(py_DG, 'python_graph.dot')
print("Graph has been successfully written to 'python_graph.dot'.")

r_dependency = pd.read_csv('Rdependency.csv')
r_DG = make_r_graph(r_dependency, min_edges=0)
print(f"\nR dependency graph has {r_DG.number_of_nodes()} nodes and {r_DG.number_of_edges()} edges.")
write_dot(r_DG, 'R_graph.dot')
print("Graph has been successfully written to 'R_graph.dot'.")
